In [6]:
import datetime
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen

import xml.etree.ElementTree as ET 

In [9]:
df = pd.read_csv('BOK_DetailTotal.csv', encoding='CP949')

In [10]:
df

,Unnamed: 0,STAT_CODE,STAT_NAME,GRP_CODE,GRP_NAME,ITEM_CODE,ITEM_NAME,CYCLE,START_TIME,END_TIME,DATA_CNT
0,0,102Y004,"1.1.1.1.1. 본원통화 구성내역(평잔, 계절조정계열)",Group1,계정항목,ABA1,"본원통화(평잔, 계절조정계열)",A,2002,2021,20
1,1,102Y004,"1.1.1.1.1. 본원통화 구성내역(평잔, 계절조정계열)",Group1,계정항목,ABA1,"본원통화(평잔, 계절조정계열)",M,200112,202207,248
2,2,102Y004,"1.1.1.1.1. 본원통화 구성내역(평잔, 계절조정계열)",Group1,계정항목,ABA104,현금통화,A,2002,2021,20
3,3,102Y004,"1.1.1.1.1. 본원통화 구성내역(평잔, 계절조정계열)",Group1,계정항목,ABA104,현금통화,M,200112,202207,248
4,4,102Y004,"1.1.1.1.1. 본원통화 구성내역(평잔, 계절조정계열)",Group1,계정항목,ABA103,중앙은행 대 예금취급기관부채,A,2002,2021,20
...,...,...,...,...,...,...,...,...,...,...,...
11212,11212,731Y006,"3.1.2.3. 원화의 대미달러, 원화의 대위안/대엔 환율",Group2,측정항목,0000100,평균자료,S,1990S1,2022S1,271
11213,11213,731Y006,"3.1.2.3. 원화의 대미달러, 원화의 대위안/대엔 환율",Group2,측정항목,0000200,말일자료,A,1990,2021,131
11214,11214,731Y006,"3.1.2.3. 원화의 대미달러, 원화의 대위안/대엔 환율",Group2,측정항목,0000200,말일자료,M,199003,202208,1634
11215,11215,731Y006,"3.1.2.3. 원화의 대미달러, 원화의 대위안/대엔 환율",Group2,측정항목,0000200,말일자료,Q,1990Q1,2022Q2,542


In [21]:
df1 = df[df['STAT_CODE'].isin(['101Y001'])] #M2 상품별 구성내역 말잔(계절조정)

In [47]:
df1.head(3)

,Unnamed: 0,STAT_CODE,STAT_NAME,GRP_CODE,GRP_NAME,ITEM_CODE,ITEM_NAME,CYCLE,START_TIME,END_TIME,DATA_CNT
140,140,101Y001,"1.1.3.1.3. M2 상품별 구성내역(말잔, 계절조정계열)",Group1,계정항목,BBGS00,"M2(말잔, 계절조정계열)",A,1970,2021,52
141,141,101Y001,"1.1.3.1.3. M2 상품별 구성내역(말잔, 계절조정계열)",Group1,계정항목,BBGS00,"M2(말잔, 계절조정계열)",M,197001,202207,631
142,142,101Y001,"1.1.3.1.3. M2 상품별 구성내역(말잔, 계절조정계열)",Group1,계정항목,BBGS01,현금통화,A,2002,2021,20


In [24]:
df2 = df1[df1['CYCLE'].isin(['M'])] 

In [48]:
df2.head(3)

,Unnamed: 0,STAT_CODE,STAT_NAME,GRP_CODE,GRP_NAME,ITEM_CODE,ITEM_NAME,CYCLE,START_TIME,END_TIME,DATA_CNT
141,141,101Y001,"1.1.3.1.3. M2 상품별 구성내역(말잔, 계절조정계열)",Group1,계정항목,BBGS00,"M2(말잔, 계절조정계열)",M,197001,202207,631
143,143,101Y001,"1.1.3.1.3. M2 상품별 구성내역(말잔, 계절조정계열)",Group1,계정항목,BBGS01,현금통화,M,200112,202207,248
145,145,101Y001,"1.1.3.1.3. M2 상품별 구성내역(말잔, 계절조정계열)",Group1,계정항목,BBGS02,요구불예금,M,200112,202207,248


In [31]:
len(df2)

11

In [7]:
key='H4MNGW6FSSUM2SSYFHJO'

In [36]:
def EcosDownload(Statname, Statcode, Freq, Begdate, Enddate, Item_Code, Subcode1, Subcode2, Subcode3):
    
    url = "https://ecos.bok.or.kr/api/StatisticSearch/"+str(key)+"/xml/kr/1/10000/%s/%s/%s/%s/%s/%s/%s/%s/"%(Statcode, Freq, Begdate, Enddate, Item_Code, Subcode1, Subcode2, Subcode3)
    print(url)

    raw = requests.get(url)
    xml = BeautifulSoup(raw.text,'xml')
    raw_data = xml.find_all("row")
    date_list = []
    value_list = []

    for item in raw_data:         
            value = item.find('DATA_VALUE').text.encode('utf-8')
            date_str = item.find('TIME').text            
            value = float(value)                
            date_list.append(datetime.datetime.strptime(date_str,'%Y%m'))
            value_list.append(value)
            
    df = pd.DataFrame(index = date_list) # Pandas 데이터프레임으로 전환합니다.    
    df['%s'%(Statname)] = value_list      
    return df

In [41]:
df_M2 = EcosDownload('M2', '101Y001', 'M', '200001', '202207', 'BBGS00', '','', '')

https://ecos.bok.or.kr/api/StatisticSearch/H4MNGW6FSSUM2SSYFHJO/xml/kr/1/10000/101Y001/M/200001/202207/BBGS00////


In [46]:
df_M2.tail(3)

,M2
2022-05-01,3700645.9
2022-06-01,3691386.1
2022-07-01,3720343.6


In [43]:
df_M2.to_csv("M2_220207.csv")